# Etude du problème d'optimisation

**Question 1.1**  

La définition de l'énergie auto-consommée (3) est interprétable de la manière suivante :  

$E^{PV}$ représente l'énergie produite par les panneaux photovoltaïques et ($w_{i}$ + $P_{i}\Delta$t) la somme des consommations non-pilotables de la maison et de l'énergie dépensée pour chauffer l'eau du ballon. Donc pour chaque i (chaque intervalle de temps) nous pouvons prendre le minimum des deux variables car nous avons trois cas :  
- Si $w_{i}$ + $P_{i}\Delta$ t > $E^{PV}_{i}$
Alors la totalité de l'énergie photovoltaïque a été consommée, on la comptabilise entièrement pour l'intervalle "i" concerné.

- Si $w_{i}$ + $P_{i}\Delta$ t < $E^{PV}_{i}$
Alors nous n'avons pas consommé la totalité de l'énergie photovoltaïque produite et le surplus de production a été revendu. Donc nous pouvons comptabiliser le terme minimal ($w_{i}$ + $P_{i}\Delta$ t) qui est égal à la consommation d'énergie photovoltaïque.

- Si $w_{i}$ + $P_{i}\Delta$ t = $E^{PV}_{i}$  
Alors peu importe le terme que nous ajoutons à la somme.  
  
La fonction min{$E^{PV}_{i}$, $w_{i}$ + $P_{i}\Delta$ t} revient à chercher :  
$E^{PV}_{i}$ < $w_{i}$ + $P_{i}\Delta$ t <=> $E^{PV}_{i}$ - $w_{i}$ < $P_{i}\Delta$ t  
En posant donc $Ê_{i}$ = $E^{PV}_{i}$ - $w_{i}$  
On remarque que les fonctions min{$E^{PV}_{i}$, $w_{i} + $P_{i}\Delta$ t} et min{$Ê_{i}$, $P_{i}\Delta$ t} sont équivalentes.  

La difficulté que comporte ces deux coûts dans le cadre d'un problème d'optimisation est que ce sont des fonctions **discontinues**.




**Question 1.2**  

Supposons que $Ê_i < P_i \Delta t$ par symétrie (h est symétrique en $Ê_i$ et $P_i \Delta t$). 

On a donc $h(Ê_i, P_i \Delta t) = \frac{Ê_i + P_i \Delta te^{- \alpha (P_i\Delta t -Ê_i)}}{1+e^{- \alpha (P_i\Delta t-Ê_i)}}$.

Or $ \alpha (P_i \Delta t -Ê_i) >>1$ donc $e^{- \alpha (P_i\Delta t -Ê_i)} <<1$.

Par développement limité à l'ordre 0 exact de la fonction $\frac{1}{1+x}$ et $1+x$, on a donc: $h(Ê_i, P_i \Delta t) = (Ê_i +0)*(1+0) = Ê_i$. Par symétrie, on a donc qu'à l'ordre 0 :
$$ h(Ê_i, P_i \Delta t) = min(Ê_i, P_i \Delta t)$$

L'avantage de la fonction h est qu'elle est $C^1$ et explicite, donc beaucoup plus facile à manipuler que la fonction minimum.

**Question 1.3**  
  

$T_{i+1}$ = $e^{-k \Delta t} * T_{i} + \frac{1 - e^{-k \Delta t}}{k}$ C(-$Q_{i} + P_{i}$)

En développant le terme $e^{-k \Delta t}$ qui devient $1-k\Delta t$ au premier ordre, on obtient l'approximation suivante pour $T_{i+1}$ :  

  
$T_{i+1}$ ~ $(1-k\Delta t)$ * $T_{i}$ + $\Delta$ t * C(-$Q_{i} + P_{i}$)   
 
Le terme $(1-k\Delta t$) * $T_{i}$ se traduit par le fait que l'on perd légèrement en température si rien n'est fait au cours du temps.  
Le terme $\Delta$ t * C(-$Q_{i} + P_{i}$) représente le fait que l'on gagne ou l'on perd en température selon si l'on a plus de puissance soustraite que de puissance de chauffage ou inversement.  
  

On peut, par une analyse dimensionnelle, souligner le fait que la constante C est homogène à une résistance thermique.


**Question 1.4**

On veut maximiser (comme prouvé en 1.2) $\sum_{i=1}^{N} h(Ê_i, P_i \Delta t)$, donc minimiser $-\sum_{i=1}^{N} h(Ê_i, P_i \Delta t)$. La seule variable sur laquelle on a un contrôle est la puissance $P$, tous les autres facteurs (production, consommation etc) sont imposés. Donc il y a une unique variable de décision (n=1), qui est $P$. On définit donc $f(P)=-\sum_{i=1}^{N} h(Ê_i, P_i \Delta t)$, qui est la fonction à minimiser. La contrainte égalité que l'on a est la relation entre température et puissance, on peut donc définir pour i entre 0 et N-1: $c_{E,i}(P)= I_{i+1} - I_i e^{-k \Delta t}- \frac {1-e^{-k \Delta t}}{k} C(-Q_i+P_i)$ . 

Les contraintes inégalités qu'on a sont $c_{I,i}(P) = T_i-T_{sat}$, $c'_{I,i}(P) = P_i-P_{M}$, $c''_{I,i}(P) = -T_{i}$, $c'''_{I,i}(P) = -P_{i}$ liées aux contraintes de sécurité.